# 02 - Clusterização: K-means e Hierárquico

Este notebook executa o pré-processamento (sem a coluna de rótulo), aplica K-means e Clusterização Hierárquica com diferentes `linkage`, avalia K-means via método do cotovelo para escolha de K, e compara os resultados entre algoritmos usando métricas de clusterização.


In [ ]:
# Configurações gerais
RANDOM_STATE = 42

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)


## Carregamento e preparação dos dados (sem rótulo)

Substitua `data/arquivo.csv` e ajuste a coluna do rótulo `target` para removê-la do `X` antes da clusterização.


In [ ]:
# Leitura
csv_path = '../data/arquivo.csv'
df = pd.read_csv(csv_path)

TARGET_COL = 'target'  # ajuste se necessário
X = df.drop(columns=[TARGET_COL]) if TARGET_COL in df.columns else df.copy()

# Inferência de tipos
feature_cols = X.columns.tolist()
dtypes = X.dtypes
numeric_features = dtypes[dtypes.kind in 'iufc'].index.tolist()
categorical_features = [c for c in feature_cols if c not in numeric_features]

print('Numéricas:', numeric_features)
print('Categóricas:', categorical_features)
print('Shape X:', X.shape)
X.head()


In [ ]:
# Pré-processamento: imputação e escala / one-hot
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler(with_mean=True, with_std=True))
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocess = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
    ],
    remainder='drop'
)

X_proc = preprocess.fit_transform(X)
print('Shape X processado:', X_proc.shape)


## Método do cotovelo para K-means

Calcula o SSE (inertia) para vários valores de K e seleciona-se o K no "cotovelo" do gráfico. Em seguida, roda-se K-means com esse K. Também computaremos métricas de qualidade de clusterização: Silhouette, Calinski-Harabasz e Davies-Bouldin.


In [ ]:
# Teste do cotovelo
K_MIN, K_MAX = 2, 12
inertias = []
Ks = list(range(K_MIN, K_MAX + 1))
for k in Ks:
    km = KMeans(n_clusters=k, n_init=10, random_state=RANDOM_STATE)
    km.fit(X_proc)
    inertias.append(km.inertia_)

plt.figure(figsize=(6,4))
plt.plot(Ks, inertias, marker='o')
plt.xlabel('K')
plt.ylabel('Inertia (SSE)')
plt.title('Método do Cotovelo - KMeans')
plt.grid(True)
plt.show()

# Escolha do K: ajuste manualmente com base no cotovelo
K_CHOSEN = 4  # ajuste após visualizar o gráfico
print('K escolhido:', K_CHOSEN)


In [ ]:
# KMeans com K escolhido
kmeans = KMeans(n_clusters=K_CHOSEN, n_init=10, random_state=RANDOM_STATE)
labels_kmeans = kmeans.fit_predict(X_proc)

sil_km = silhouette_score(X_proc, labels_kmeans)
ch_km = calinski_harabasz_score(X_proc, labels_kmeans)
db_km = davies_bouldin_score(X_proc, labels_kmeans)

print(f'Silhouette (KMeans): {sil_km:.4f}')
print(f'Calinski-Harabasz (KMeans): {ch_km:.2f}')
print(f'Davies-Bouldin (KMeans): {db_km:.4f}')


## Clusterização Hierárquica

Rodaremos `AgglomerativeClustering` com o mesmo K escolhido no cotovelo e variaremos dois métodos de linkage (por exemplo, `ward` e `complete`), comparando as métricas com o K-means.


In [ ]:
def avaliar_clusterizacao(labels, nome):
    sil = silhouette_score(X_proc, labels)
    ch = calinski_harabasz_score(X_proc, labels)
    db = davies_bouldin_score(X_proc, labels)
    print(f"\n{nome}")
    print(f"Silhouette: {sil:.4f}")
    print(f"Calinski-Harabasz: {ch:.2f}")
    print(f"Davies-Bouldin: {db:.4f}")

# Dois linkages para comparar
for linkage in ['ward', 'complete']:
    # 'ward' requer métrica euclidiana
    if linkage == 'ward':
        model = AgglomerativeClustering(n_clusters=K_CHOSEN, linkage=linkage)
    else:
        model = AgglomerativeClustering(n_clusters=K_CHOSEN, linkage=linkage, metric='euclidean')

    labels_hier = model.fit_predict(X_proc)
    avaliar_clusterizacao(labels_hier, f'Hierárquico ({linkage})')


## Comparação e Métrica Própria

Além das métricas clássicas, podemos adotar uma métrica composta simples para ranquear as soluções: maximize `silhouette` e `calinski-harabasz`, minimize `davies-bouldin`. Uma proposta: score = z(silhouette) + z(calinski) - z(davies). Abaixo, exemplificamos como calcular e comparar para K-means e Hierárquico.


In [ ]:
from sklearn.preprocessing import StandardScaler as _Std

# Coletar métricas
results = []
results.append({'alg':'KMeans', 'sil': sil_km, 'ch': ch_km, 'db': db_km})

# Recalcular para hierárquicos com os mesmos linkages para registrar
scores_hier = {}
for linkage in ['ward', 'complete']:
    if linkage == 'ward':
        model = AgglomerativeClustering(n_clusters=K_CHOSEN, linkage=linkage)
    else:
        model = AgglomerativeClustering(n_clusters=K_CHOSEN, linkage=linkage, metric='euclidean')
    labels = model.fit_predict(X_proc)
    scores_hier[linkage] = {
        'sil': silhouette_score(X_proc, labels),
        'ch': calinski_harabasz_score(X_proc, labels),
        'db': davies_bouldin_score(X_proc, labels)
    }
    results.append({'alg': f'Hier({linkage})', **scores_hier[linkage]})

# Score composto (z-normalizado)
df_res = pd.DataFrame(results)
scaler = _Std()
z_sil = scaler.fit_transform(df_res[['sil']]).ravel()
z_ch  = scaler.fit_transform(df_res[['ch']]).ravel()
z_db  = scaler.fit_transform(df_res[['db']]).ravel()

df_res['score_composto'] = z_sil + z_ch - z_db
print(df_res.sort_values('score_composto', ascending=False))
